In [ ]:
import shutil
from IPython.display import clear_output
!pip install telethon
!pip install nest_asyncio
clear_output()
print('Libas installed')

In [42]:
from telethon import TelegramClient, functions, types
import pandas as pd
import numpy as np
import asyncio
import nest_asyncio

# Применение nest_asyncio для текущего цикла событий
nest_asyncio.apply()

api_id =
api_hash =

In [ ]:
# подключаемся  и ищем нужный чат
async def main():
    async with TelegramClient('test_tg', api_id, api_hash) as client:
        dialogs = await client.get_dialogs()
        for dialog in dialogs:
            print(f"ID: {dialog.id}, Title: {dialog.title}")

# Запуск асинхронной функции в уже работающем цикле событий
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [ ]:
# ID: -1002121762239, Title: ДыпломЧат
# ID: -1002116531534, Title: GoodGather GRS

In [ ]:
# Получаем все сообщения из чата по его ID - проверка
# async def main():
#     async with TelegramClient('test_tg', api_id, api_hash) as client:

#         async for message in client.iter_messages(-1002121762239):
#             print(f"ID: {message.id}, Date: {message.date}, Message: {message.text}")

# loop = asyncio.get_event_loop()
# loop.run_until_complete(main())

In [103]:
async def main():
    async with TelegramClient('test_tg', api_id, api_hash) as client:
        messages = []
        async for message in client.iter_messages(-1002121762239):
            sender = await message.get_sender()
            # if sender is not None:
            messages.append({
                'id': message.id,
                'date': message.date,
                'text': message.text,
                'reply_to_msg_id': message.reply_to_msg_id,
                'from_id': sender.id if sender else 'creator',
                'from_username': sender.username if sender else 'creator',
                # 'from_first_name': sender.first_name,
                # 'from_last_name': sender.last_name if sender.last_name else None
            })


        df = pd.DataFrame(messages)
        df.to_csv('messages.csv', index=False)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [104]:
df = pd.read_csv('messages.csv')

In [105]:
df

,id,date,text,reply_to_msg_id,from_id,from_username
0,2354,2024-05-21 07:17:29+00:00,👍,2342.0,899084599,Ira_ins
1,2353,2024-05-20 19:19:31+00:00,👍🏼,2342.0,724536482,katrinoushakova
2,2352,2024-05-20 19:15:14+00:00,👍🏻,2342.0,413509637,kskhristenko
3,2351,2024-05-20 19:06:47+00:00,👍🏼,2342.0,942064159,kostrovaksu
4,2349,2024-05-20 18:55:11+00:00,👍🏼,2342.0,985222560,NaN
...,...,...,...,...,...,...
1613,482,2024-05-05 17:35:42+00:00,NaN,NaN,2116531534,onlyfunsdiploma
1614,481,2024-05-05 17:35:42+00:00,NaN,NaN,2116531534,onlyfunsdiploma
1615,480,2024-05-05 17:35:42+00:00,NaN,NaN,2116531534,onlyfunsdiploma
1616,479,2024-05-05 17:35:42+00:00,100\nПогулять\nГород\n**Секретный особняк в це...,NaN,2116531534,onlyfunsdiploma


In [106]:
# удаляем сообщения которые технические
df = df.drop(df[df.text.isna() & df.reply_to_msg_id.isna()].index, axis=0)
df.shape

(1447, 6)

In [140]:
# удаляем сообщения которые писала я
df = df.drop(df[df.from_username == 'creator'].index, axis=0)
df.shape

(1307, 10)

In [107]:
df.isna().sum()

id                   0
date                 0
text                10
reply_to_msg_id    269
from_id              0
from_username      173
dtype: int64

In [108]:
df['length'] = df.text.apply(lambda x: len(x) if pd.notna(x) else x)

In [109]:
df.loc[df.text.str.contains('Место', na=False)]

,id,date,text,reply_to_msg_id,from_id,from_username,length
8,2342,2024-05-20 18:44:01+00:00,"61\nЭкскурсия\nУличная, История\n**""Вокруг да ...",NaN,2116531534,onlyfunsdiploma,643.0
22,2328,2024-05-17 19:09:51+00:00,"32\nФестиваль, Лекция\nОбразовательное, Психол...",NaN,2116531534,onlyfunsdiploma,575.0
24,2326,2024-05-17 19:09:39+00:00,"42\nВыставка\nИстория, Искусство\n**В Музее Мо...",NaN,2116531534,onlyfunsdiploma,1100.0
31,2319,2024-05-17 18:58:17+00:00,74\nМастерклассы\nРастения\n**Икебана WorkShop...,NaN,2116531534,onlyfunsdiploma,857.0
56,2294,2024-05-17 17:04:30+00:00,69\nСпектакль\nНовое\n**Спектакль «Комната №7»...,NaN,2116531534,onlyfunsdiploma,551.0
...,...,...,...,...,...,...,...
1589,529,2024-05-05 17:38:19+00:00,110\nКонцерт\nПесни\n**Егор Крид**\n\nГлавный ...,NaN,2116531534,onlyfunsdiploma,809.0
1598,504,2024-05-05 17:37:39+00:00,"107\nВыставка\nФотография, Спорт, История\n**С...",NaN,2116531534,onlyfunsdiploma,867.0
1603,494,2024-05-05 17:36:20+00:00,103\nВыставка\nЖивотные\n**Гладим экзотических...,NaN,2116531534,onlyfunsdiploma,562.0
1608,487,2024-05-05 17:35:52+00:00,101\nВыставка\nСовременное Искусство\n**Тексти...,NaN,2116531534,onlyfunsdiploma,656.0


In [110]:
df.loc[df.text.str.contains('Место', na=False), 'help'] = df.loc[
    df.text.str.contains('Место', na=False), 'text'
    ].str.split('\n')

In [111]:
df['poi_id'] = np.NaN
df['poi_id'] = df.loc[
    df.text.str.contains('Место', na=False), 'text'
    ].str.split('\n').str[0]

In [112]:
df.loc[df.help.notna(), 'help'] = df.loc[df.help.notna(), 'help'].apply(lambda x: [item.replace('*', '').strip() for item in x if isinstance(item, str) and item.strip() != ''])

In [113]:
df['desc'] = np.NaN
df['desc'] = df.loc[
    df.text.str.contains('Место', na=False), 'help'
    ].str[4]

In [114]:
df[df.id == 2342]

,id,date,text,reply_to_msg_id,from_id,from_username,length,help,poi_id,desc
8,2342,2024-05-20 18:44:01+00:00,"61\nЭкскурсия\nУличная, История\n**""Вокруг да ...",NaN,2116531534,onlyfunsdiploma,643.0,"[61, Экскурсия, Уличная, История, ""Вокруг да о...",61,Прогулка с москвоведом по Никитскому бульвару ...


In [116]:
# NaN - лизок
df[df.reply_to_msg_id == 2342]

,id,date,text,reply_to_msg_id,from_id,from_username,length,help,poi_id,desc
0,2354,2024-05-21 07:17:29+00:00,👍,2342.0,899084599,Ira_ins,1.0,NaN,NaN,NaN
1,2353,2024-05-20 19:19:31+00:00,👍🏼,2342.0,724536482,katrinoushakova,2.0,NaN,NaN,NaN
2,2352,2024-05-20 19:15:14+00:00,👍🏻,2342.0,413509637,kskhristenko,2.0,NaN,NaN,NaN
3,2351,2024-05-20 19:06:47+00:00,👍🏼,2342.0,942064159,kostrovaksu,2.0,NaN,NaN,NaN
4,2349,2024-05-20 18:55:11+00:00,👍🏼,2342.0,985222560,NaN,2.0,NaN,NaN,NaN
5,2347,2024-05-20 18:54:28+00:00,👍🏻,2342.0,778456547,annsayten,2.0,NaN,NaN,NaN


In [131]:
df[df.from_username.isna() & df[df.from_username.isna()].text.str.contains('а', na=False)]

,id,date,text,reply_to_msg_id,from_id,from_username,length,help,poi_id,desc
70,2280,2024-05-16 05:04:04+00:00,"👍🏼\nКрасивые места, очень живописно, привлекае...",2272.0,985222560,NaN,129.0,NaN,NaN,NaN
253,2093,2024-05-14 18:30:12+00:00,👎\nНе нравится концепция заведений где так экс...,2085.0,985222560,NaN,180.0,NaN,NaN,NaN
259,2087,2024-05-14 18:28:08+00:00,С зайками на постоянной основе играют \nС откр...,2085.0,985222560,NaN,64.0,NaN,NaN,NaN
274,2072,2024-05-14 18:20:55+00:00,👎\nОписание мне не понравилось \nКажется будет...,2068.0,985222560,NaN,167.0,NaN,NaN,NaN
279,2067,2024-05-14 18:18:02+00:00,👎\nДумаю выставка не произведет большого впеча...,2059.0,985222560,NaN,171.0,NaN,NaN,NaN
286,2060,2024-05-14 18:12:46+00:00,"Никто же не знает наверняка, а вдруг мы по при...",2056.0,985222560,NaN,67.0,NaN,NaN,NaN
292,2054,2024-05-14 18:10:15+00:00,"👎\nМожем и дома посмотреть в своей компании, з...",2051.0,985222560,NaN,56.0,NaN,NaN,NaN
302,2044,2024-05-14 18:01:00+00:00,Поехали,2042.0,985222560,NaN,7.0,NaN,NaN,NaN
326,2018,2024-05-14 15:50:24+00:00,"👍🏼\nПокушать всегда двумя руками за, а если ещ...",2016.0,985222560,NaN,125.0,NaN,NaN,NaN
354,1989,2024-05-13 19:29:55+00:00,👍🏼\nДа да да\nНа такое надо сходить \nНасладит...,1981.0,985222560,NaN,73.0,NaN,NaN,NaN


In [132]:
id_to_poi = dict(zip(df[df.poi_id.notna()]['id'], df[df.poi_id.notna()]['poi_id']))

# Применение сопоставления к столбцу 'reply_to_msg_id' и заполнение 'poi_id'
df['poi_id'] = df['reply_to_msg_id'].map(id_to_poi)

In [144]:
df['react'] = np.NaN
df['react'] = df[df.text.notna()].text.apply(lambda x: 1 if '👍' in x else -1 if '👎' in x else 0)

In [148]:
df[df.reply_to_msg_id == 2342]

,id,date,text,reply_to_msg_id,from_id,from_username,length,help,poi_id,desc,react
0,2354,2024-05-21 07:17:29+00:00,👍,2342.0,899084599,Ira_ins,1.0,NaN,61,NaN,1.0
1,2353,2024-05-20 19:19:31+00:00,👍🏼,2342.0,724536482,katrinoushakova,2.0,NaN,61,NaN,1.0
2,2352,2024-05-20 19:15:14+00:00,👍🏻,2342.0,413509637,kskhristenko,2.0,NaN,61,NaN,1.0
3,2351,2024-05-20 19:06:47+00:00,👍🏼,2342.0,942064159,kostrovaksu,2.0,NaN,61,NaN,1.0
4,2349,2024-05-20 18:55:11+00:00,👍🏼,2342.0,985222560,NaN,2.0,NaN,61,NaN,1.0
5,2347,2024-05-20 18:54:28+00:00,👍🏻,2342.0,778456547,annsayten,2.0,NaN,61,NaN,1.0


In [154]:
df[['text', 'react']].value_counts()[:20]

text      react
👍🏼         1.0     156
👍🏻         1.0     110
👍          1.0     108
👎         -1.0      99
👎🏻        -1.0      96
👎🏼        -1.0      38
Все        0.0      17
Я все      0.0      13
Готово     0.0       9
я все      0.0       7
Я тут      0.0       6
Тут        0.0       4
все        0.0       2
тут        0.0       2
и я все    0.0       2
готово     0.0       2
✅          0.0       2
ахахахах   0.0       2
И я тут    0.0       2
ой         0.0       1
Name: count, dtype: int64

In [155]:
user_map = {
    'kostrovaksu': 'Ксю',
    'annsayten': 'Аня',
    np.NaN: 'Лиза',
    'katrinoushakova': 'Катя',
    'Ira_ins': 'Ира',
    'kskhristenko': 'Окси',
    'onlyfunsdiploma': 'onlyfunsdiploma'
    }

df.from_username = df.from_username.map(user_map)

In [156]:
df[df.reply_to_msg_id == 2342]

,id,date,text,reply_to_msg_id,from_id,from_username,length,help,poi_id,desc,react
0,2354,2024-05-21 07:17:29+00:00,👍,2342.0,899084599,Ира,1.0,NaN,61,NaN,1.0
1,2353,2024-05-20 19:19:31+00:00,👍🏼,2342.0,724536482,Катя,2.0,NaN,61,NaN,1.0
2,2352,2024-05-20 19:15:14+00:00,👍🏻,2342.0,413509637,Окси,2.0,NaN,61,NaN,1.0
3,2351,2024-05-20 19:06:47+00:00,👍🏼,2342.0,942064159,Ксю,2.0,NaN,61,NaN,1.0
4,2349,2024-05-20 18:55:11+00:00,👍🏼,2342.0,985222560,Лиза,2.0,NaN,61,NaN,1.0
5,2347,2024-05-20 18:54:28+00:00,👍🏻,2342.0,778456547,Аня,2.0,NaN,61,NaN,1.0


In [176]:
interactions = df[df.react != 0].pivot_table(index='from_username', columns='poi_id', values='react')
assert interactions.min().min() == -1
assert interactions.max().max() == 1
interactions.to_excel('interactions_df.xlsx', index=False)

In [ ]:
# вытаскиваем новые взаимодействия

df1 = pd.read_excel('interactions_df.xlsx')  # старая версия взаимождействий
# df2 = # новая версия выгрузки чата

# обрабатываем также новую верию

# Найти строки, которые присутствуют в df2, но отсутствуют в df1
diff = pd.concat([df2, df1]).drop_duplicates(keep=False)
# diff - новые взаимодействия